In [ ]:
import mlrun
import nuclio
import os

In [ ]:
# nuclio: ignore

In [ ]:
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_API=${V3IO_API}

In [ ]:
%%nuclio config 
spec.build.baseImage = "mlrun/mlrun:0.6.5-rc15"

In [ ]:
%nuclio config kind="serving"
%nuclio cmd python -m pip install tensorflow==2.2.0
%nuclio cmd python -m pip install keras

In [35]:
import pandas as pd
from mlrun import MLClientCtx
import os
import mlrun
import numpy as np
import tensorflow
import json

In [39]:
class RNN_Model_Serving(mlrun.serving.V2ModelServer):    
    def load(self):
        """load and initialize the model and/or other elements"""
        model_file,extra_data = self.get_model(suffix = ".h5")
        self.model = tensorflow.keras.models.load_model(model_file)
        
    def predict(self, body):
        try:
            """Generate model predictions from sample."""
            feats = np.asarray(body['inputs'])
            result = self.model.predict(feats)
            result = json.dumps(result.tolist())
            return result
        except Exception as e:
            raise Exception("Failed to predict %s" % e)

In [40]:
# nuclio: end-code

In [ ]:
myclass = RNN_Model_Serving(context = context,name = "rnn_serving",model_path="/User/test/demos/stock-analysis/models/mymodel.h5")
myclass.load()

In [5]:
import mlrun
import os 
fn = mlrun.code_to_function("RNN_Model_Serving",kind="serving")
fn.apply(mlrun.auto_mount())

In [2]:
fn.add_model("model2",class_name="RNN_Model_Serving", model_path = "/User/test/demos/stock-analysis/models/mymodel.h5")

In [6]:
addr = fn.deploy(project = "stocks-" + os.getenv('V3IO_USERNAME'))

> 2021-07-12 05:54:35,845 [info] Starting remote function deploy
2021-07-12 05:54:35  (info) Deploying function
2021-07-12 05:54:35  (info) Building
2021-07-12 05:54:36  (info) Staging files and preparing base images
2021-07-12 05:54:36  (info) Building processor image
2021-07-12 05:54:37  (info) Build complete
2021-07-12 05:54:58  (info) Function deploy complete
> 2021-07-12 05:54:59,114 [info] function deployed, address=default-tenant.app.dev39.lab.iguazeng.com:31269


In [56]:
server = fn.to_mock_server()

> 2021-07-01 10:14:59,310 [info] model model2 was loaded
> 2021-07-01 10:14:59,311 [info] Loaded ['model2']


In [57]:
x = np.array([[6.9955170e-01, 6.9952875e-01, 2.7922913e-02, 2.7853036e-02,
6.9955170e-01, 7.0086759e-01, 7.0118028e-01, 7.0142627e-01,
2.7922913e-02, 0.0000000e+00, 0.0000000e+00],
[6.9955170e-01, 6.9998503e-01, 1.6527303e-03, 2.7853036e-02,
7.0000792e-01, 7.0085293e-01, 7.0118028e-01, 7.0203447e-01,
1.6527303e-03, 0.0000000e+00, 0.0000000e+00],
[6.9955170e-01, 7.0025057e-01, 1.6904050e-04, 2.7853036e-02,
7.0027345e-01, 7.0014298e-01, 7.0190376e-01, 7.0128226e-01,
1.6904050e-04, 0.0000000e+00, 0.0000000e+00],
[6.9955170e-01, 7.0144778e-01, 1.6904050e-04, 2.7853036e-02,
7.0147055e-01, 7.0178574e-01, 7.0236105e-01, 7.0295709e-01,
7.3906886e-03, 0.0000000e+00, 0.0000000e+00],
[6.9955170e-01, 7.0324355e-01, 1.6904050e-04, 2.7853036e-02,
7.0326620e-01, 7.0308524e-01, 7.0490342e-01, 7.0427048e-01,
2.4815742e-03, 0.0000000e+00, 0.0000000e+00],
[6.9955170e-01, 7.0324355e-01, 1.6904050e-04, 2.7853036e-02,
7.0191067e-01, 7.0173001e-01, 7.0354480e-01, 7.0291305e-01,
2.9976186e-03, 0.0000000e+00, 0.0000000e+00],
[6.9955170e-01, 7.0324355e-01, 1.6904050e-04, 2.7853036e-02,
7.0166123e-01, 7.0148063e-01, 7.0284635e-01, 7.0249581e-01,
2.7904075e-03, 0.0000000e+00, 0.0000000e+00],
[6.9955170e-01, 7.0324355e-01, 1.6904050e-04, 2.7853036e-02,
7.0133996e-01, 7.0143080e-01, 7.0297277e-01, 7.0250750e-01,
4.1491759e-04, 0.0000000e+00, 0.0000000e+00],
[6.9955170e-01, 7.0324355e-01, 1.6904050e-04, 2.7853036e-02,
7.0150572e-01, 7.0251614e-01, 7.0281982e-01, 7.0370042e-01,
2.1256472e-03, 0.0000000e+00, 0.0000000e+00],
[6.9955170e-01, 7.0324355e-01, 1.6904050e-04, 2.7853036e-02,
7.0272487e-01, 7.0258951e-01, 7.0429617e-01, 7.0376801e-01,
1.4207334e-03, 0.0000000e+00, 0.0000000e+00]]).reshape(1,10,11).tolist()

In [58]:
server.test("/v2/models/model2/infer", {"inputs": x})

{'id': '7d68eea27a9447f8a46e96ef8c0bb04e',
 'model_name': 'model2',
 'outputs': '[[0.7206476926803589]]'}